# Neural Network Approach

In [2]:
import keras
from keras.layers import Dense
from keras.models import Sequential

/home/wangmk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np

In [5]:
from keras import metrics
from keras.callbacks import EarlyStopping

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
from keras.utils import to_categorical

In [78]:
from sklearn.metrics import roc_auc_score as roc
from sklearn.metrics import roc_curve

In [112]:
from sklearn.model_selection import train_test_split

In [7]:
alldata = pd.read_csv('/home/wangmk/UM/Research/SURE/cleaned_data/full_cleaned_data.csv')
alldata['Agree'] = ~(alldata['True_state']^alldata['Alert'])

In [8]:
alldata.head()

,Trial,True_state,Alert,Identification,First_toggle_time,RMS,Attention,Click,Top_Left_Enemy,Top_Right_Enemy,Bottom_Left_Enemy,Bottom_Right_Enemy,Top_Left_Dark,Top_Right_Dark,Bottom_Left_Dark,Bottom_Right_Dark,Agree
0,0,True,True,True,4179,65.244487,0.914573,2,True,False,False,False,False,True,True,True,True
1,1,False,False,False,3653,67.293907,0.895000,2,False,False,False,False,True,True,True,True,True
2,2,False,True,False,2186,100.119892,0.854271,2,False,False,False,False,True,False,True,False,False
3,3,False,False,False,3412,33.403009,0.889447,2,False,False,False,False,True,False,True,True,True
4,4,False,False,False,4514,70.063858,0.929648,2,False,False,False,False,True,True,True,True,True


In [135]:
abnormal = alldata[alldata['RMS']>60]
normal = alldata[alldata['RMS']<=60]

In [157]:
newdata = abnormal.append(normal.sample(n=160,random_state=40))
newdata['performance'] = newdata['RMS'] <= 60

In [158]:
newdata['First_toggle_time'] = newdata['First_toggle_time']/10000

In [159]:
X = newdata.drop(['RMS','performance','Click'],axis=1)
y = newdata['performance']

In [13]:
X.shape[1]

15

In [151]:
model = Sequential()
model.add(Dense(20, activation='relu', input_shape = (15,)))
model.add(Dense(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [152]:
early_stopping_monitor = EarlyStopping(patience=3)

In [153]:
y_neuralnets = to_categorical(y)
model.fit(X,y_neuralnets,epochs=30,validation_split=0.3,callbacks=[early_stopping_monitor])

Train on 224 samples, validate on 96 samples
Epoch 1/30
224/224 [==============================] - 2s 7ms/step - loss: 0.6915 - acc: 0.6071 - val_loss: 1.2180 - val_acc: 0.0625
Epoch 2/30
224/224 [==============================] - 0s 170us/step - loss: 0.6719 - acc: 0.6875 - val_loss: 0.9975 - val_acc: 0.0208
Epoch 3/30
224/224 [==============================] - 0s 172us/step - loss: 0.6461 - acc: 0.7098 - val_loss: 0.9118 - val_acc: 0.0000e+00
Epoch 4/30
224/224 [==============================] - 0s 245us/step - loss: 0.6333 - acc: 0.7143 - val_loss: 1.0361 - val_acc: 0.0000e+00
Epoch 5/30
224/224 [==============================] - 0s 205us/step - loss: 0.6186 - acc: 0.7143 - val_loss: 1.0727 - val_acc: 0.0000e+00
Epoch 6/30
224/224 [==============================] - 0s 196us/step - loss: 0.6089 - acc: 0.7143 - val_loss: 1.0560 - val_acc: 0.0000e+00


In [173]:
accuracy = []
auc = []

In [174]:
states = np.array([2,5,10,50,70,90,120,400,1000,2000])

In [175]:
for state in states:
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=state,stratify=y)
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape = (15,)))
    model.add(Dense(10, activation='relu'))
    #model.add(Dense(10, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    y_train = to_categorical(y_train)
    model.fit(X_train, y_train,epochs=30,validation_split=0.2,callbacks=[early_stopping_monitor])
    y_pred = model.predict(X_test)
    auc.append(roc(y_test,y_pred[:,1]))

Train on 204 samples, validate on 52 samples
Epoch 1/30
204/204 [==============================] - 3s 14ms/step - loss: 5.9024 - acc: 0.5196 - val_loss: 7.1571 - val_acc: 0.4231
Epoch 2/30
204/204 [==============================] - 0s 263us/step - loss: 5.6520 - acc: 0.5196 - val_loss: 6.8396 - val_acc: 0.4231
Epoch 3/30
204/204 [==============================] - 0s 261us/step - loss: 5.2733 - acc: 0.5196 - val_loss: 6.2264 - val_acc: 0.4231
Epoch 4/30
204/204 [==============================] - 0s 234us/step - loss: 4.6772 - acc: 0.5098 - val_loss: 5.1645 - val_acc: 0.4231
Epoch 5/30
204/204 [==============================] - 0s 239us/step - loss: 3.6465 - acc: 0.5049 - val_loss: 3.5174 - val_acc: 0.4615
Epoch 6/30
204/204 [==============================] - 0s 262us/step - loss: 2.3607 - acc: 0.4951 - val_loss: 1.8814 - val_acc: 0.4423
Epoch 7/30
204/204 [==============================] - 0s 267us/step - loss: 1.2005 - acc: 0.4755 - val_loss: 0.7397 - val_acc: 0.4808
Epoch 8/30
204/204

Epoch 11/30
204/204 [==============================] - 0s 244us/step - loss: 0.6824 - acc: 0.5686 - val_loss: 0.7488 - val_acc: 0.4615
Epoch 12/30
204/204 [==============================] - 0s 250us/step - loss: 0.6754 - acc: 0.5931 - val_loss: 0.7375 - val_acc: 0.4615
Epoch 13/30
204/204 [==============================] - 0s 241us/step - loss: 0.6661 - acc: 0.5833 - val_loss: 0.7157 - val_acc: 0.4615
Epoch 14/30
204/204 [==============================] - 0s 221us/step - loss: 0.6648 - acc: 0.5882 - val_loss: 0.7129 - val_acc: 0.4423
Epoch 15/30
204/204 [==============================] - 0s 251us/step - loss: 0.6665 - acc: 0.5882 - val_loss: 0.7070 - val_acc: 0.4808
Epoch 16/30
204/204 [==============================] - 0s 247us/step - loss: 0.6598 - acc: 0.6127 - val_loss: 0.7168 - val_acc: 0.5000
Epoch 17/30
204/204 [==============================] - 0s 238us/step - loss: 0.6601 - acc: 0.6225 - val_loss: 0.7036 - val_acc: 0.5000
Epoch 18/30
204/204 [==============================] - 

204/204 [==============================] - 0s 276us/step - loss: 0.6952 - acc: 0.5833 - val_loss: 0.8068 - val_acc: 0.5192
Train on 204 samples, validate on 52 samples
Epoch 1/30
204/204 [==============================] - 3s 14ms/step - loss: 1.9234 - acc: 0.4755 - val_loss: 1.2609 - val_acc: 0.5192
Epoch 2/30
204/204 [==============================] - 0s 256us/step - loss: 1.1068 - acc: 0.4363 - val_loss: 0.6855 - val_acc: 0.5962
Epoch 3/30
204/204 [==============================] - 0s 245us/step - loss: 0.7156 - acc: 0.4706 - val_loss: 0.6997 - val_acc: 0.5385
Epoch 4/30
204/204 [==============================] - 0s 260us/step - loss: 0.7770 - acc: 0.5049 - val_loss: 0.7361 - val_acc: 0.5385
Epoch 5/30
204/204 [==============================] - 0s 248us/step - loss: 0.7559 - acc: 0.5147 - val_loss: 0.6722 - val_acc: 0.5769
Epoch 6/30
204/204 [==============================] - 0s 238us/step - loss: 0.6971 - acc: 0.5441 - val_loss: 0.6469 - val_acc: 0.5769
Epoch 7/30
204/204 [=========

In [172]:
1-np.array(auc)

array([0.58691406, 0.62890625, 0.65527344, 0.55957031, 0.58691406,
       0.6484375 , 0.63574219, 0.57128906, 0.69238281, 0.65332031])

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=90,stratify=y)
#y_train = to_categorical(y_train)
y_train = to_categorical(y_train)
model.fit(X_train, y_train,epochs=30,callbacks=[early_stopping_monitor])
y_pred = model.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred[:,0])

Epoch 1/30
224/224 [==============================] - 1s 6ms/step - loss: 1.5144 - acc: 0.5402
Epoch 2/30
224/224 [==============================] - 0s 209us/step - loss: 0.7996 - acc: 0.5000
Epoch 3/30
224/224 [==============================] - 0s 180us/step - loss: 0.7222 - acc: 0.5580
Epoch 4/30
224/224 [==============================] - 0s 192us/step - loss: 0.7333 - acc: 0.5536
Epoch 5/30
224/224 [==============================] - 0s 154us/step - loss: 0.6710 - acc: 0.5938
Epoch 6/30
224/224 [==============================] - 0s 131us/step - loss: 0.6725 - acc: 0.6205
Epoch 7/30


/home/wangmk/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


224/224 [==============================] - 0s 209us/step - loss: 0.6669 - acc: 0.6116
Epoch 8/30
224/224 [==============================] - 0s 155us/step - loss: 0.6676 - acc: 0.6116
Epoch 9/30
224/224 [==============================] - 0s 149us/step - loss: 0.6579 - acc: 0.6161
Epoch 10/30
224/224 [==============================] - 0s 152us/step - loss: 0.6574 - acc: 0.6205
Epoch 11/30
224/224 [==============================] - 0s 142us/step - loss: 0.6485 - acc: 0.6250
Epoch 12/30
224/224 [==============================] - 0s 141us/step - loss: 0.6503 - acc: 0.6295
Epoch 13/30
224/224 [==============================] - 0s 144us/step - loss: 0.6570 - acc: 0.6161
Epoch 14/30
224/224 [==============================] - 0s 145us/step - loss: 0.6452 - acc: 0.6473
Epoch 15/30
224/224 [==============================] - 0s 155us/step - loss: 0.6464 - acc: 0.5938
Epoch 16/30
224/224 [==============================] - 0s 287us/step - loss: 0.6412 - acc: 0.6429
Epoch 17/30
224/224 [=============

In [146]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred[:,0])

In [134]:
y_pred[:,0]

array([5.2781093e-01, 9.6576536e-01, 4.9527165e-01, 7.8700435e-01,
       3.9921245e-01, 3.5112223e-01, 9.9916542e-01, 9.7007430e-01,
       1.4523604e-07, 1.0000000e+00, 2.3089497e-01, 8.6935169e-01,
       1.0064879e-01, 9.9999309e-01, 8.3719021e-01, 3.3655077e-01,
       9.9711955e-01, 7.1858209e-01, 8.2754129e-01, 1.4469554e-02,
       8.5599221e-02, 1.8053779e-01, 5.2807128e-01, 9.9986053e-01,
       4.6354693e-01, 2.5273836e-01, 9.4924819e-01, 1.1996118e-05,
       9.9115258e-01, 9.9188954e-01, 9.9693966e-01, 9.9981266e-01,
       5.6586689e-01, 2.0685905e-01, 2.4357524e-03, 1.0780092e-02,
       1.0451220e-03, 2.5900088e-02, 5.6790650e-01, 7.7393037e-01,
       5.9709424e-01, 5.6371850e-01, 7.7456957e-01, 5.6853676e-01,
       8.5441127e-02, 3.2835554e-02, 1.9453971e-01, 3.7325958e-12,
       9.9571770e-01, 9.2963201e-01, 1.0000000e+00, 3.0722255e-03,
       5.5444032e-01, 2.9924141e-08, 4.1801161e-01, 9.8528636e-01,
       1.0000000e+00, 7.5642481e-02, 5.1396197e-01, 9.9999011e

In [147]:
roc(y_test, y_pred[:,0])

0.4748263888888889